# Statistical Results

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from scipy import stats
from IPython.display import HTML, display
import moss
import lyman

## Data ingest

In [2]:
subjects = {}
subjects["dots"] = lyman.determine_subjects(["dots_subjects"])
subjects["sticks"] = lyman.determine_subjects(["sticks_subjects"])
subjects["rest"] = lyman.determine_subjects(["rest_subjects"])

In [3]:
decoding_ftemp = "decoding_analysis/{}_{}_{}.pkz"
decoding_results = {}
for roi in ["ifs", "mfc"]:
    for exp in ["dots", "sticks"]:
        for subj in subjects[exp]:
            key = subj, exp, roi
            decoding_results[key] = moss.load_pkl(decoding_ftemp.format(*key))

In [4]:
spatial_ftemp = "spatial_analysis/{}_{}_{}.pkz"
spatial_results = {}
for exp in ["dots", "sticks"]:
    for subj in subjects[exp]:
        key = subj, exp, "ifs"
        spatial_results[key] = moss.load_pkl(spatial_ftemp.format(*key))

In [5]:
correlation_ftemp = "correlation_analysis/{}_{}_{}.pkz"
correlation_results = {}
for exp in ["dots", "sticks", "rest"]:
    for subj in subjects[exp]:
        key = subj, exp, "ifs"
        correlation_results[key] = moss.load_pkl(correlation_ftemp.format(*key))

In [6]:
def field_vector(results, field, exp, roi="ifs", func=None):
    """Pull out a data vector from a list of results objects."""
    out = {}
    subj_list = subjects[exp]
    for subj in subj_list:
        val = getattr(results[(subj, exp, roi)], field)
        if func is not None:
            val = func(val)
        out[subj] = val
    return pd.Series(out, name=field)

---

## Results formatting

In [7]:
def title(text, level=4):
    display(HTML("<h{}>{}</h{}>".format(level, text, level)))

def mean_pm_sd(vector, fmt="", label=""):
    m = ("{:" + fmt + "}").format(np.mean(vector))
    sd = ("{:" + fmt + "}").format(np.std(vector))
    text = "{} {} &plusmn; {}".format(label, m, sd)
    display(HTML(text))

def ttest(dof, t, p, label=""):
    text = "{} <em>t</em>({}) = {:.2f}; <em>P</em> = {:.3g}".format(label, dof, t, p)
    display(HTML(text))

def sig_subs(pctiles, label="", thresh=95):
    sig = (pctiles > thresh).sum()
    tot = len(pctiles)
    text = "{} {}/{} subjects <em>P</em> &lt; 0.05".format(label, sig, tot)
    display(HTML(text))

----

## Decoding analyses

### Group tests against chance

In [8]:
for exp in ["dots", "sticks"]:
    for roi in ["ifs", "mfc"]:
        title("Experiment: {} ({})".format(exp.capitalize(), roi.upper()))
        accs = field_vector(decoding_results, "acc", exp, roi)
        chance = field_vector(decoding_results, "chance", exp, roi)
        mean_pm_sd(accs, ".1%", "Accuracy")
        t, p = stats.ttest_1samp(accs - chance, 0)
        dof = len(accs) - 1
        ttest(dof, t, p)

### Individual subject tests against chance

In [9]:
for exp in ["dots", "sticks"]:
    for roi in ["ifs", "mfc"]:
        title("Experiment: {} ({})".format(exp.capitalize(), roi.upper()))
        acc_pctiles = field_vector(decoding_results, "acc_pctile", exp, roi)
        sig_subs(acc_pctiles)

### Group tests between ROIs

In [10]:
for exp in ["dots", "sticks"]:
    title("Experiment: " + exp.capitalize())
    ifs = field_vector(decoding_results, "acc", exp, "ifs")
    mfc = field_vector(decoding_results, "acc", exp, "mfc")
    t, p = stats.ttest_rel(ifs, mfc)
    dof = len(ifs) - 1
    ttest(dof, t, p)

### Context preference cluster sizes

In [11]:
for exp in ["dots", "sticks"]:
    title("Experiment: " + exp.capitalize())
    intersect_point = field_vector(spatial_results, "intersect", exp, func=lambda v: v[0])
    mean_pm_sd(intersect_point, ".2f", "Cluster size: ")

----

## Subnetwork analyses

### MDS variance explained

In [12]:
for exp in ["dots", "sticks", "rest"]:
    title("Experiment: " + exp.capitalize())
    varexp = field_vector(correlation_results, "mds_varexp", exp)
    mean_pm_sd(varexp, ".1%", "MDS variance explained: ")

### Tail correlation strength: group tests

In [13]:
for exp in ["dots", "sticks", "rest"]:
    title("Experiment: " + exp.capitalize())
    same = field_vector(correlation_results, "tail_corrs", exp, func=lambda v: v[0])
    diff = field_vector(correlation_results, "tail_corrs", exp, func=lambda v: v[1])
    ratio = same / diff
    mean_pm_sd(ratio, ".2f", "Ratio: ")
    t, p = stats.ttest_rel(same, diff)
    dof = len(same) - 1
    ttest(dof, t, p)

### Tail correlation strength: subject tests

In [14]:
for exp in ["dots", "sticks", "rest"]:
    title("Experiment: " + exp.capitalize())
    pctiles = field_vector(correlation_results, "corr_pctile", exp)
    sig_subs(pctiles, thresh=97.5)

### Similarity of correlation structures

In [15]:
def upper_tri(mat):
    idx = np.triu_indices_from(mat, 1)
    return mat[idx]

In [16]:
subj_rs = pd.Series(dtype=np.float)
subj_mantels = pd.Series(dtype=np.float)
rs = np.random.RandomState(sum(map(ord, "mantel")))
for subj in subjects["sticks"]:

    task = correlation_results[(subj, "sticks", "ifs")].corrmat
    rest = correlation_results[(subj, "rest", "ifs")].corrmat

    r, p = stats.pearsonr(upper_tri(task), upper_tri(rest))
    subj_rs[subj] = r
    
    null_dist = []
    for _ in xrange(100):
        n = len(task)
        perm = rs.permutation(np.arange(n))
        task_triu = upper_tri(task[perm][:, perm])
        rest_triu = upper_tri(rest)
        null_dist.append(stats.pearsonr(task_triu, rest_triu)[0])
    pctile = stats.percentileofscore(null_dist, r)
    subj_mantels[subj] = pctile    

In [17]:
m, sd = np.mean(subj_rs), np.std(subj_rs)
t, p = stats.ttest_1samp(subj_rs, 0)
dof = len(subj_rs) - 1
mean_pm_sd(subj_rs, ".2f", "Correlation matrix similarity: ")
ttest(dof, t, p)
sig_subs(subj_mantels, "Mantel test: ")

### Subnetworks strength in first resting state run

In [18]:
subj_pctiles = field_vector(correlation_results, "corr_times_pctiles", "rest",
                            func=lambda v: v[0])
sig_subs(subj_pctiles)

### Long-range subnetworks

In [19]:
for exp in ["dots", "sticks"]:
    for dim in ["2D", "3D"]:
        title("Experiment: " + exp.capitalize() + " ({})".format(dim))
        subj_pctiles = field_vector(correlation_results, "corr_distance_pctiles", exp,
                                    func=lambda v: v[dim][-1])
        sig_subs(subj_pctiles)